In [ ]:
%pip install tensorflow
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
  print('Found {} GPUs.'.format(len(gpus)))
else:
  print('No GPUs found.')

DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


2024-05-11 09:27:53.328101: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 09:27:54.678627: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 1 GPUs.


In [ ]:
! pip install pandas
! pip install numpy
! pip install matplotlib
! pip install scikit-learn
! pip install --upgrade pip setuptools wheel
! pip install opencv-python

ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.9 MB/s eta 0:00:00
  Using cached setuptools-69.5.1-py3-none-any.whl (894 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from PIL import ImageFont
import warnings
warnings.filterwarnings('ignore')

import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import pandas as pd
from sklearn.model_selection import train_test_split

import cv2
# from google.colab.patches import cv2_imshow

import tensorflow as tf

from tensorflow.keras.layers import Conv2D, MaxPool2D

In [ ]:
imgs_title = []
imgs_title.extend(list(os.listdir('/media/ssn/d/IMAGE_CLEF_MED/train_images/train2')))

FileNotFoundError: [Errno 2] No such file or directory: '/media/ssn/d/IMAGE_CLEF_MED/train_images/train2'

In [ ]:
imgs_title[:10]

In [ ]:
#df -----> image id and cui id
concepts_df = pd.read_csv('/media/ssn/d/IMAGE_CLEF_MED/train_concepts_manual.csv',usecols=[0,1])
print(concepts_df.head(5))
print(concepts_df.shape)
imgname_CUI_mappings={}
for index,row in concepts_df.iterrows():
  imgname_CUI_mappings[row['ID']+'.jpg']=str(row['CUIs']).split(';')



In [ ]:
#df1 ------> cui id and concept
CUI_mapping_df = pd.read_csv('/media/ssn/d/IMAGE_CLEF_MED/cui_mapping.csv',usecols=[0,1])
print(CUI_mapping_df.head(5))
print(CUI_mapping_df.shape)
CUI_name_mapping={}
for index,row in CUI_mapping_df.iterrows():
  CUI_name_mapping[row['CUI']]=row['Canonical name']
print(CUI_name_mapping)

        CUI                   Canonical name
0  C0016522            Foramen Ovale, Patent
1  C0457846  Cervical segment of spinal cord
2  C0578537               Cavitation of lung
3  C0179751                Epidural catheter
4  C3489393                    Hiatal Hernia
(1934, 2)
{'C0016522': 'Foramen Ovale, Patent', 'C0457846': 'Cervical segment of spinal cord', 'C0578537': 'Cavitation of lung', 'C0179751': 'Epidural catheter', 'C3489393': 'Hiatal Hernia', 'C0730166': 'Bone structure of fifth metacarpal', 'C1261077': 'Structure of left lower lobe of lung', 'C0152443': 'Urethral diverticulum', 'C0227301': 'Descending part of duodenum', 'C0031354': 'Pharyngeal structure', 'C4039172': 'Posterior point of nasal spine', 'C0227387': 'Left colic flexure', 'C0227385': 'Hepatic flexure of colon', 'C0228126': 'Leptomeninges', 'C0230436': 'Popliteal fossa structure', 'C0037922': 'Spinal Canal', 'C1260954': 'Morphologically altered structure', 'C0450184': 'Jejunal loop', 'C0729936': 'Cardiac chamb

In [ ]:
imgs_title.sort()
for title in imgs_title:
  if title not in imgname_CUI_mappings.keys():
    imgs_title.remove(title)

In [ ]:
img_list=[]
for img_title in imgs_title:
  img_loaded = tf.keras.utils.load_img('/media/ssn/d/IMAGE_CLEF_MED/train_images/train2/'+img_title)
  img_array=img_to_array(img_loaded)
  img_resized=cv2.resize(img_array, (32, 32))
  img_list.append(img_resized)

In [ ]:
CUI_train=[imgname_CUI_mappings[img] for img in imgs_title]
CUI_train[:5]

[['C0040405'], ['C0041618'], ['C0040405'], ['C0002978'], ['C0040405']]

In [ ]:
print(img_list[:5])
print(CUI_train[:5])

[array([[[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [255., 255., 255.]],

       ...,

       [[255., 255., 255.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', padding='same', input_shape=img_list[0].shape))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())25
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(19, activation='sigmoid'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 16)        448       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 16)        64        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 16)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 16, 16, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        4640      
                                                                 
 batch_normalization_1 (Bat  (None, 16, 16, 32)        1

2024-05-11 09:38:53.279667: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9270 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5


                                                                 
 batch_normalization_5 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 19)                2451      
                                                                 
Total params: 142771 (557.70 KB)
Trainable params: 141907 (554.32 KB)
Non-trainable params: 864 (3.38 KB)
_________________________________________________________________


In [ ]:
train_array=np.array(img_list)
# y_train_array=np.array(y_train)
print(train_array.shape)

(60163, 32, 32, 3)


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Assuming y_train is a list of lists containing multiple labels for each sample
# Example: y_train = [['class1', 'class2'], ['class2', 'class3'], ...]
mlb = MultiLabelBinarizer()

# Fit and transform the labels
train_encoded = mlb.fit_transform(CUI_train)

# Check the shape of the encoded labels
print("Shape of encoded labels:", train_encoded.shape)


Shape of encoded labels: (60163, 19)


In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics=['accuracy'])
es_callback = keras.callbacks.EarlyStopping (monitor= 'val_loss', patience=3)
history = model.fit(train_array, train_encoded, epochs=20)

2024-05-11 09:38:54.727293: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 739282944 exceeds 10% of free system memory.
2024-05-11 09:38:55.342148: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 739282944 exceeds 10% of free system memory.


Epoch 1/20


2024-05-11 09:38:58.458730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-05-11 09:39:00.698710: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-05-11 09:39:00.704641: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 7.5
2024-05-11 09:39:00.704702: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2024-05-11 09:39:00.704861: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1881/1881 [==============================] - 107s 53ms/step - loss: 0.1889 - accuracy: 0.4462
Epoch 2/20
1881/1881 [==============================] - 99s 53ms/step - loss: 0.1161 - accuracy: 0.5581
Epoch 3/20
1881/1881 [==============================] - 103s 55ms/step - loss: 0.1070 - accuracy: 0.5886
Epoch 4/20
1881/1881 [==============================] - 100s 53ms/step - loss: 0.1015 - accuracy: 0.6035
Epoch 5/20
1881/1881 [==============================] - 98s 52ms/step - loss: 0.0976 - accuracy: 0.6125
Epoch 6/20
1881/1881 [==============================] - 103s 55ms/step - loss: 0.0945 - accuracy: 0.6241
Epoch 7/20
1881/1881 [==============================] - 103s 55ms/step - loss: 0.0925 - accuracy: 0.6244
Epoch 8/20
1881/1881 [==============================] - 99s 53ms/step - loss: 0.0911 - accuracy: 0.6287
Epoch 9/20
1881/1881 [==============================] - 98s 52ms/step - loss: 0.0885 - accuracy: 0.6373
Epoch 10/20
1881/1881 [==============================] - 101s 54ms/ste

In [ ]:
imgs_title_test = []
imgs_title_test.extend(list(os.listdir('/media/ssn/d/IMAGE_CLEF_MED/test_images/test')))

In [ ]:
img_list_test=[]
for img_title in imgs_title_test:
    img_loaded = tf.keras.utils.load_img('/media/ssn/d/IMAGE_CLEF_MED/test_images/test/'+img_title)
    img_array=img_to_array(img_loaded)
    img_resized=cv2.resize(img_array, (32, 32))
    img_list_test.append(img_resized)

In [ ]:
img_list_test=img_list_test[0]
print(type(img_list_test))

<class 'numpy.ndarray'>


In [ ]:
y_pred=model.predict(np.array(img_list_test))
print(len(y_pred[1]))

312/312 [==============================] - 3s 10ms/step
19


In [ ]:
#df -----> image id and cui id
concepts_df_test = pd.read_csv('/media/ssn/d/IMAGE_CLEF_MED/test_concepts.csv',usecols=[0,1])
print(concepts_df_test.head(5))
print(concepts_df_test.shape)
imgname_CUI_mappings_test={}
for index,row in concepts_df_test.iterrows():
  imgname_CUI_mappings_test[row['ID']+'.jpg']=str(row['CUIs']).split(';')
print(imgname_CUI_mappings_test['ROCOv2_2023_test_008996.jpg'])



In [ ]:
CUI_test=[imgname_CUI_mappings_test[img] for img in imgs_title_test]
CUI_test[:5]

[['C0002978', 'C0205097', 'C1261287', 'C0034052', 'C0226032', 'C0027051'],
 ['C0024485', 'C0235946', 'C0007799', 'C0012359'],
 ['C0002978', 'C1947917'],
 ['C0040405', 'C0038257'],
 ['C1306645', 'C1140618', 'C1999039']]

In [ ]:
from sklearn.metrics import accuracy_score

# Assuming y_test is a list of lists containing multiple labels for each sample
# Example: y_test = [['class1', 'class2'], ['class2', 'class3'], ...]

# Encode y_test using the same MultiLabelBinarizer used for y_train
y_test_encoded = mlb.transform(CUI_test)

# Assuming you have computed y_pred, which is the model's predictions
# Example: y_pred = model.predict(x_test_array)

# Round y_pred to obtain binary predictions
y_pred_binary = (y_pred > 0.5).astype(int)
#print(y_pred_binary)
# Calculate accuracy
accuracy = accuracy_score(y_test_encoded, y_pred_binary)

print("Accuracy:", accuracy)


Accuracy: 0.5604693140794224


In [ ]:
print(y_test_encoded.shape)
print(y_pred_binary.shape)

(9972, 19)
(9972, 19)


In [ ]:
# Convert encoded y_test back to original labels
y_test_original = mlb.inverse_transform(y_test_encoded)

# Convert encoded y_pred_binary back to original labels
y_pred_original = mlb.inverse_transform(y_pred_binary)


In [ ]:
model.save('/media/ssn/d/IMAGE_CLEF_MED/task1Model.h5')

: 